In [141]:
import os
import pandas as pd
import numpy as np
import regex as re
from datetime import datetime, timedelta
import statsmodels.api as sm
import statsmodels.formula.api as smf

import ast

# Source code downloaded
from vincenty import vincenty_inverse
from tqdm import tqdm
pd.set_option('display.max_columns', 100)

# Cleaning EPA Data - Creating the condensed .csv files

In [145]:
current_dir = os.getcwd()
NOAA_dir = current_dir + r'\NOAA Weather Data'
EPA_dir = current_dir + r'\EPA Ozone Data'

In [146]:
zip_files = os.listdir(EPA_dir + r'\Raw EPA Data')


In [147]:
def create_ozone_id(statelist, countylist, sitelist):
    """
    Helper function for creating ozoneID's based on an EPA dataset
    
    returns: 3 lists appended together into tuples to be added into a column
    """
    return [(a,b,c) for a, b, c in zip(statelist, countylist, sitelist)]

def append_ozone_id(ozone_df):
    """
    Creates a copy of the dataframe and adds a new column that concatenates the state code, county code, and site number
    into a tuple to make an individual identifier for each ozone reporting location in that year. 
    
    returns: dataframe with ozoneID
    """
    temp = ozone_df.copy() 
    temp["ozoneID"] = create_ozone_id(temp["State Code"], temp["County Code"], temp["Site Num"])
    return temp

In [148]:
time_period = ['09:00', '10:00', '11:00', '12:00', '13:00', '14:00', 
                      '15:00', '16:00', '17:00', '18:00', '19:00', '20:00']

def monitor_day_filter(grouped_dataframe):
    """
    Used in split-apply-combine after grouping by monitor-day in order to filter out days that have 
    less than 9 observations in the hours from 9am to 9pm
    """
    times = grouped_dataframe["Time Local"]
    indicator = [time in time_period for time in times]
    return sum(indicator) >= 9

## Construct 2 measures of ozone concentrations at the monitor-day level

- Daily Maximum: Groupby date, then return the maximum of that day

- Daily 8-hour Maximum: Groupby date, then average hours 0-8, 8-4, 4-12, return the maximum of that 

- Disqualify all monitor-days for which observations are not recorded for at least 9 hours between 9AM and 9PM, disqualify all monitors that have less than 75% of the days recorded from June1 to August 31,

- Disqualify monitors that are in counties close to other counties that have more stringent regulation (?????) 

Do this in one group by, write an apply function to return both of these as a new data frame per date. 

In [149]:
def classify_time(time_str):
    time_int = int(time_str[0:2])
    if time_int < 8:
        return 1
    elif 8 <= time_int < 16:
        return 2
    elif  16 <= time_int:
        return 3
    else:
        raise ValueError('Time Local date was invalid?') 

def calculate_max_8hrmax(grouped_dataframe):
    """
    Used in split-apply-combine after grouping by monitor-day - calculate the maximum of the day as well as the 
    8 hour maximum - 8 hour maximum 
    """
       
    grouped_dataframe["Time Chunk"] = grouped_dataframe["Time Local"].apply(classify_time)
    
    samples_all = grouped_dataframe["Sample Measurement"]
    
    mean_1 = np.mean(grouped_dataframe.loc[grouped_dataframe["Time Chunk"] == 1, "Sample Measurement"])
    mean_2 = np.mean(grouped_dataframe.loc[grouped_dataframe["Time Chunk"] == 2, "Sample Measurement"])
    mean_3 = np.mean(grouped_dataframe.loc[grouped_dataframe["Time Chunk"] == 3, "Sample Measurement"])
    
    # CHANGE THIS TO MAX FUCK
    daily_max = np.max(samples_all)
    eight_hour_max = max(mean_1, mean_2, mean_3)
    
    output = grouped_dataframe.iloc[[0],:]
    
    output["Daily Max Ozone"] = daily_max
    output["Daily 8hr Mean Ozone"] = eight_hour_max
    return output
                        

In [150]:
@np.vectorize
def is_summer(date_str):
    """
    Checks if a dat str (in the format of the tablse) is in the summer 
    """
    month = int(re.search(r'\d{4}-(\d{2})-\d{2}', date_str).group(1))
    return 6 <= month <= 8


def monitor_year_filter(grouped_ozone_id):
    """ 
    Used in split-apply-combine after grouping by monitor in order to filter out entire monitors that do not have 
    observations in 25% or more during the summer ozone months Do this after calculating the maximum and 8hr maximums 
    
    DO THIS BEFORE APPLYING MAX DATE BECAUSE MAX DATE SLOW AS HELL 
    """
    # 92 days between June 1 and August 31, need 75% observations or more == only accept if greater than or equal to 69
    dates = grouped_ozone_id["Date Local"].unique()
    return sum(is_summer(dates)) >= 69
   
    

## For Loop Instructions 
- Read the csv, then append ozone_id to get the ozone_Ids
- First group by date and ozone_id and filter out monitor-dates using monitor-day-filter 
    - This will net us with a dataframe that only has monitor dates for enough observations in the 9 hours  
- Apply the calculate max-8hrmax function to grouped monitor-days 
    - This will result with each day ending up justb being a single observation with 2 new columns. 
- Now group by monitor and look at whether or not this year had enough values, return final data frame after filtering

In [ ]:
zip_files = os.listdir(EPA_dir + r'\Raw EPA Data')
zip_files

for filename in zip_files: 
    
    year = filename[13:17]
    
    if False:
#     if f"filtered_{year}_EPA.csv" in os.listdir(EPA_dir + r'\Filtered EPA Data'):
        print(f"Skipping {year}. already done")
        continue
    else: 
        print(f"working on {year}")
        filepath = EPA_dir + r"\\Raw EPA Data\\" + filename 
        temp = pd.read_csv(filepath)
        print("read csv done!")
        
        temp = append_ozone_id(temp)
        print(f"finished appending for {filename}")
        
        temp = temp.groupby(["ozoneID", "Date Local"]).filter(monitor_day_filter)
        print(f"finished dayfilter for {filename}")
        
        temp = temp.groupby("ozoneID").filter(monitor_year_filter)
        print(f"finished yearfilter for {filename}")
        
        temp = temp.groupby(["ozoneID", "Date Local"], group_keys=False).apply(calculate_max_8hrmax)
        print(f"finished {filename}, exporting to csv!")

        path = EPA_dir + r'\Filtered EPA Data\filtered_' + year + "_EPA.csv"
        temp.to_csv(path)

working on 1989


C:\Users\Danny\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (17,23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


read csv done!
finished appending for hourly_44201_1989.zip
finished dayfilter for hourly_44201_1989.zip
finished yearfilter for hourly_44201_1989.zip


Note: (13, 85, 1) has 48 counts for some reason - it's just duplicated values can ignore because not gonna affect mean and 8hr mean

### Creating the lists of state_code county_code pairs that need to be deleted  

In [ ]:
county_list = pd.read_stata(current_dir + "\Author Data\AER20090377_CountyList.dta")
county_list["fips"] = county_list["fips"].astype(int)
county_list = county_list.rename(columns = {"state_code": "State Code", "county_code":"County Code"})
county_list = county_list.drop(columns = "county_desc")
county_list.head()

In [ ]:
neighbor_list = pd.read_stata(current_dir + "\Author Data\AER20090377_NeighborData.dta")
neighbor_list.head()

In [ ]:
# If deleting too many values, then make this so that it only counts the neighbors that are == 1
neighbor_fips = neighbor_list["fips"].values

In [ ]:
def get_urban_df(year):
    """
    Returns the dataframe of the urban designations as extracted from "Getting Urban Designation.ipynb"|
    """
    assert 1989 <= year <= 2003, "Bad year input"
    df = pd.read_csv(current_dir + r"\Author Data\county_urban_designation\county_urban_designation_" + str(year) + ".csv")
    df["urban"] = [int(text[1]) for text in df["urban"]]
    return df
    

### Filtering out the above counties for each file, selecting only the summer months, and then also storing necessary data to create the summary table, also adding rural/urban/suburban

- First add all of county_list data to the normal data
- Filter based on that 
- Apply county filter from above
- Store the number of remaining ozone_locations
- Store the number of monitor-days left
- Count how many are rural/urban 

In [ ]:
statistics = {}

In [ ]:
filtered_dir_list = os.listdir(EPA_dir + r'\Filtered EPA Data')
filtered_files = [file for file in filtered_dir_list if file.endswith(".csv")]

In [ ]:
summer_output_folder_path = EPA_dir + r'\Filtered EPA Data\onlysummer\\' 
second_level_cleaning_folder_path = EPA_dir + r'\Filtered EPA Data\second_cleaning\\' 

In [ ]:
for df_name in filtered_files:
    
    year = int(df_name[9:13])
    
    if False:
#     if r"summer_only_EPA_" + str(year) + ".csv" in os.listdir(summer_output_folder_path):
        print(r"summer_only_EPA_" + str(year) + ".csv already exists, skipping")
        continue
    else:
        
        year = int(df_name[9:13])
        
        print("working on " + str(year))
        temp = pd.read_csv(EPA_dir + r'\Filtered EPA Data\\' + df_name, low_memory = False)

        
        # IF THERE IS CANADA, DELETE THOSE ROWS LMAO 
        
        if "CC" in temp["State Code"].unique(): 
            temp = temp[temp["State Code"] != "CC"]
            temp["State Code"] = temp["State Code"].astype(int)
            
        
        # add information from county_list file -- add fips 
        temp = temp.merge(county_list.iloc[:,0:3], on = ["State Code", "County Code"]) # now we have a bunch of extra info but that's okay 

        # remove counties that are in the neighbors list 
        neighbor_indicator = [county_fip not in neighbor_fips for county_fip in temp["fips"]]
        temp = temp[neighbor_indicator] 


        # add urban designation - for missing observations with NaN, fill with zero.
        # 3 = suburban, 2 = rural, 1 = urban
        temp = temp.merge(get_urban_df(year), on = "ozoneID", how = "left").fillna({"urban":0})     
        
        temp = temp.drop(columns = ["Parameter Code", "POC", "Datum", "Parameter Name", 
                                    "Time Local", "Date GMT", "Time GMT", "Units of Measure", 
                                    "Uncertainty", "Qualifier", "Method Type", "Method Code", 
                                    "Method Name", "MDL", "Date of Last Change", "Time Chunk"])

        temp.to_csv(second_level_cleaning_folder_path + r"condensed_EPA_" + str(year) + ".csv")

#         # select only observations that are in the months of June, July, August 
#         summer_indicator = [int(date[5:7]) in [6, 7, 8] for date in temp["Date Local"]]
#         summer_only = temp[summer_indicator]

#         summer_only.to_csv(summer_output_folder_path + r"summer_only_EPA_" + str(year) + ".csv")

        #get statistics

#         num_obs = summer_only.shape[0]
#         num_counties = len(summer_only["fips"].unique())
#         num_monitors = len(summer_only["ozoneID"].unique())
#         urban_0_count = sum(summer_only.groupby("ozoneID")["urban"].unique() == 0)
#         urban_1_count = sum(summer_only.groupby("ozoneID")["urban"].unique() == 1)
#         urban_2_count = sum(summer_only.groupby("ozoneID")["urban"].unique() == 2)
#         urban_3_count = sum(summer_only.groupby("ozoneID")["urban"].unique() == 3)

#         statistics[year] = {"Observations":num_obs,
#                             "Counties":num_counties,
#                             "Total Monitors":num_monitors,
#                             "Urban_1":urban_1_count,
#                             "Urban_2":urban_2_count,
#                             "Urban_3":urban_3_count,
#                             "Urban_0":urban_0_count,
#                            }
            
    

In [ ]:
# summary_table = pd.DataFrame(statistics).T
# summary_table.to_csv("summary_table2.csv")
# summary_table

In [ ]:
pd.read_csv("summary_table2.csv", index_col = 0)

# Cleaning NOAA Data + Joining with EPA Data

In [ ]:
current_dir = os.getcwd()
NOAA_dir = current_dir + r'\NOAA Weather Data'
EPA_dir = current_dir + r'\EPA Ozone Data'

filtered_EPA_dir = current_dir + r'/EPA Ozone Data/Filtered EPA Data/second_cleaning/'


In [ ]:
filtered_EPA_files = [file for file in os.listdir(filtered_EPA_dir) if file.endswith(".csv")]

In [ ]:
NOAA_files = [file for file in os.listdir(NOAA_dir) if file.endswith(".csv.gz")]

In [ ]:
us_station_codes = pd.read_csv(NOAA_dir + r"\us_station_codes.csv", index_col = 0).drop(columns = ["a", "b"])

In [ ]:
def get_ozoneID_coords(ozone_df):
    """
    Groups entire dataframe by ozoneID and then applies lambda function that extracts the first entry of Latitude, Longitude
    
    returns: a series indexed by ozoneID that gives back information that can then be indexed into using key's 'Latitude' and
    'Longitude'
    
    NOTE: WE CAN GET ALL UNIQUE OZONE ID FROM THIS OUTPUT'S INDEX using output.index
    """
    #first check that the required columns are there, otherwise print an error
    if all(column in ozone_df.columns for column in ["ozoneID", "Latitude", "Longitude"]):
        return ozone_df.groupby("ozoneID").apply(lambda gr: gr[["Latitude", "Longitude"]].iloc[0,:])
    else:
        raise Exception("one of the columns needed in ozoneID, Latitude, Longitude was missing")

In [ ]:
from vincenty import vincenty_inverse

def get_closest_stations(lat_long_pair, NOAA_info):
#     print("working on " + str(lat_long_pair))
    temp = NOAA_info.copy()
    temp["ozone_lat"] = lat_long_pair["Latitude"]
    temp["ozone_long"] = lat_long_pair["Longitude"]
    temp["vincenty_dist"] = [vincenty_inverse((a, b), (c, d)) for a, b, c, d in zip(temp["lat"], 
                                                                            temp["long"], 
                                                                            temp["ozone_lat"], 
                                                                            temp["ozone_long"])]
    sorted_distances = temp.sort_values("vincenty_dist")[["StationId","vincenty_dist"]].iloc[0:10, :]
    return sorted_distances.values.tolist()



def get_closest_stations_euclidian(lat_long_pair, NOAA_info):
#     print("working on " + str(lat_long_pair))
    temp = NOAA_info.copy()
    temp["ozone_lat"] = lat_long_pair["Latitude"]
    temp["ozone_long"] = lat_long_pair["Longitude"]
    temp["euclidian_dist"] = np.sqrt((np.array(temp["lat"]) - np.array(temp["ozone_lat"]))**2 + (np.array(temp["long"]) - np.array(temp["ozone_long"]))**2)
    
    sorted_distances = temp.sort_values("euclidian_dist")["StationId"].iloc[0:10]
    return sorted_distances.values.tolist()

# Clean and Join NOAA data to filtered EPA Data

In [ ]:
# finished code

joined_data_dir = current_dir + r'/Joined Data/'

headers = ["StationId", "Date", "Measurement", "Value", "Flag1", "Flag2", "Flag3", "Flag4"]
closest_station_col_name = dict(zip(range(10), ["station_" + str(i) for i in range(10)]))

for i in range(len(filtered_EPA_files)):
    EPA_file = filtered_EPA_files[i]
#     print(EPA_file)
    NOAA_file = NOAA_files[i]
#     print(NOAA_file)
    year = NOAA_file[0:4]
    
    if year + "EPA_NOAA_joined.csv" in os.listdir(joined_data_dir):
        print("Skipping " + year + ", already filtered data.")
        continue
    else:
        print("Working on " + year )
        
        print("Reading csvs")
        EPA_data = pd.read_csv(filtered_EPA_dir + EPA_file, low_memory = False, index_col = 0).drop(columns = ["Unnamed: 0.1"])
        NOAA_data = pd.read_csv(NOAA_dir + "\\" + NOAA_file, names = headers)

        
        print("Cleaning NOAA")

        # clean NOAA_data
        NOAA_data = NOAA_data[NOAA_data["StationId"].str.startswith("US")] # selecting only US data

 
        NOAA_data = NOAA_data[NOAA_data["Measurement"].str.contains("TMAX|TMIN|PRCP|SNOW")].drop(columns = ["Flag1", "Flag2", "Flag3", "Flag4"])# selecting only important values and dropping uncessesary rows

        NOAA_data["Datetime"] = pd.to_datetime(NOAA_data["Date"], format='%Y%m%d', errors='ignore') # adding datetime to NOAA_data 

        unique_station_ids = pd.DataFrame({"StationId":NOAA_data["StationId"].unique()})
        unique_station_ids = unique_station_ids.merge(us_station_codes, on = "StationId", how = "left").drop(columns = ["elev", "name"])

        
        # get closest stations of the available stations 
        closest_stations = {}
        ozone_ID_coords = get_ozoneID_coords(EPA_data)
        unique_ozone_ids = ozone_ID_coords.index

        print("Getting closest stations")
        for i in tqdm(range(len(unique_ozone_ids))):
            # check station ids for each of the entries
            ozone_id = unique_ozone_ids[i]
            ozone_station_coord = ozone_ID_coords.loc[ozone_id]
            #euclidian distance much faster but can use vincenty - just not vectorized 
            closest_stations[ozone_id] = get_closest_stations_euclidian(ozone_station_coord, unique_station_ids)

        closest_station_df = pd.DataFrame(closest_stations).T.rename(columns = closest_station_col_name)

        # start finding closest stations and figuring out the weathers

        EPA_data["Datetime"] = pd.to_datetime(EPA_data["Date Local"], format='%Y-%m-%d', errors='ignore')
        merged_EPA_data = EPA_data.merge(closest_station_df, left_on = "ozoneID", right_index = True)
        merged_EPA_data
    
        print("Getting weather values")
        for var in ["TMAX", "TMIN", "SNOW", "PRCP"]:
            #select smaller subset to use 
            need_weather = merged_EPA_data[["Datetime"] + ["station_" + str(i) for i in range(10)]]

            # work with one NOAA variable at a time
            NOAA_variable_data = NOAA_data[NOAA_data["Measurement"] == var].rename(columns = {"Value": var}).drop(columns = ["Measurement", "Date"])

            for i in tqdm(range(10)): 
                current_station = "station_" + str(i) 
                current_variable = var + "_" + str(i)
                temp = need_weather.merge(NOAA_variable_data, left_on = ["Datetime", current_station], right_on = ["Datetime", "StationId"],  how = "left")\
                    .drop(columns = ["StationId"])\
                    .rename(columns = {var:current_variable})
                need_weather = temp

            NOAA_values = need_weather.iloc[:, -10:].values.tolist()
            no_na_NOAA_vals = [[value for value  in row if value  == value ] for row in NOAA_values]
            EPA_data[var] = no_na_NOAA_vals

        print("exporting")
        EPA_data.to_csv(joined_data_dir + year + "EPA_NOAA_joined.csv")

    
    

    

# Adding on Indicators for Implementation of Regulation

Using data provided in the author's online appendix, we create indicators for which regulations were applied where

In [ ]:
current_dir = os.getcwd()
NOAA_dir = current_dir + r'\NOAA Weather Data'
EPA_dir = current_dir + r'\EPA Ozone Data'

joined_data_dir = current_dir + r'/Joined Data/'


In [ ]:
joined_csv_files = [file for file in os.listdir(joined_data_dir) if file.endswith(".csv")]

In [ ]:
treatment_indicators = pd.read_csv("treatment_indicators.csv")

In [ ]:
def get_first_value(x):
    temp = ast.literal_eval(x)
    if len(temp) > 0:
        return temp[0]
    else:
        return temp

for file in joined_csv_files:
    print("Working on " + file)
    if "final_" + file in os.listdir(current_dir + "/Final Data"):
        print(file + " already joined, skipping")
        continue
    else:
        temp_df = pd.read_csv(joined_data_dir + file, index_col = 0)
        # this step takes the longest - gets the first value from each of the list of weather variables
        temp_df[["TMAX", "TMIN", "SNOW", "PRCP"]] = temp_df[["TMAX", "TMIN", "SNOW", "PRCP"]].applymap(get_first_value)
        temp_df["Datetime"] = pd.to_datetime(temp_df["Datetime"])
        temp_df["year"] = pd.DatetimeIndex(temp_df["Datetime"]).year
        temp_df["month"] = pd.DatetimeIndex(temp_df["Datetime"]).month
        merged_df = temp_df.merge(treatment_indicators, on = ["fips", "month", "year"], how = "left")
        merged_df.to_csv(current_dir + "/Final Data/" + "final_" + file)
        
        
        

# Finishing Data Cleaning - Merging All Data

In [ ]:
final_data_dir = current_dir + r'/Final Data/' 

In [ ]:
final_csv_files = [file for file in os.listdir(final_data_dir) if file.endswith(".csv")]

In [ ]:
join_together = []
for file in final_csv_files:
    join_together.append(pd.read_csv(final_data_dir + file, index_col = 0))

In [ ]:
# final = pd.concat(join_together)

In [ ]:
# final

In [ ]:
state_code_join = [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 
                   28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49,
                   50, 51, 53, 54, 55, 56]

census_region_join = [3, 4, 4, 3, 4, 4, 1, 3, 3, 3, 3, 4, 4, 2, 2, 2, 2, 3, 3, 1, 3, 1, 2, 2, 3, 2, 4, 2, 4, 1, 1, 4,
              1, 3, 2, 2, 3, 4, 1, 1, 3, 2, 3, 3, 4, 1, 3, 4, 3, 2, 4]

state_census_map = dict(zip(state_code_join, census_region_join))

In [ ]:
# add census region

final["census_region"] = final["State Code"].map(state_census_map)

# remove mexico

indicator = [state_code in state_code_join for state_code in final["State Code"]]

final_data = final[indicator]



In [ ]:
# final_data.to_csv("final_cleaned_data.csv")


# Reproducing Author's Results

In [3]:
final_data = pd.read_csv('final_cleaned_data.csv', index_col = 0)

C:\Users\Danny\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (16,17,18,19,22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Danny\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
# created lagged temperature
final_data["Datetime"] = pd.to_datetime(final_data["Datetime"])
final_data["day_before"] = [date - timedelta(days=1) for date in final_data["Datetime"]]
lagged_weather = final_data[["TMAX", "TMIN", "SNOW", "PRCP", "ozoneID", "Datetime"]].rename(columns = {"TMAX":"lagged_TMAX", "TMIN":"lagged_TMIN", "SNOW":"lagged_SNOW", "PRCP":"lagged_PRCP", "Datetime":"day_before"})
merged_lagged_weather = final_data.merge(lagged_weather, on = ["ozoneID", "day_before"]).rename(columns = {"Daily Max Ozone":"mean_ozone", "Daily 8hr Mean Ozone":"8hr_mean_ozone"})


In [5]:
summer_months = [6, 7, 8]
summer_only_indicator = [month in summer_months for month in merged_lagged_weather["month"]]
summer_only = merged_lagged_weather.copy()[summer_only_indicator]

In [6]:
summer_only["ln_ozone"] = np.log(summer_only["mean_ozone"])
summer_only["ln_8hr_ozone"] = np.log(summer_only["8hr_mean_ozone"])
summer_only = summer_only.replace([np.inf, -np.inf], np.nan)

C:\Users\Danny\Anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
summer_only.head()

,State Code,County Code,Site Num,Latitude,Longitude,Date Local,Sample Measurement,State Name,County Name,ozoneID,mean_ozone,8hr_mean_ozone,fips,urban,Datetime,TMAX,TMIN,SNOW,PRCP,year,month,RVPStart,RVPEnd,RFGStart,RFGEnd,RegFlag,RVPI,treat_rvpII,treat_rfg,treat_rvpI,treat_CARB,TreatRFG,panelid,RFGStart2,RFGEnd2,TreatRVPII,RVPStart2,RVPEnd2,TreatCARB,TreatRFGCA,RVPCty,RFGCty,CARBCty,TreatRVPca,census_region,day_before,lagged_TMAX,lagged_TMIN,lagged_SNOW,lagged_PRCP,ln_ozone,ln_8hr_ozone
91,1,73,1003,33.485556,-86.915,1989-06-01,0.002,Alabama,Jefferson,"(1, 73, 1003)",0.025667,0.049375,1073,2.0,1989-06-01,328.0,217.0,0.0,0.0,1989,6,NaN,NaN,NaN,NaN,0.0,10.5,0.0,0.0,1.0,0.0,0.0,12.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1989-05-31,328.0,217.0,0.0,0.0,-3.662562,-3.008311
92,1,73,1003,33.485556,-86.915,1989-06-02,0.002,Alabama,Jefferson,"(1, 73, 1003)",0.012250,0.025875,1073,2.0,1989-06-02,311.0,200.0,0.0,79.0,1989,6,NaN,NaN,NaN,NaN,0.0,10.5,0.0,0.0,1.0,0.0,0.0,12.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1989-06-01,328.0,217.0,0.0,0.0,-4.402229,-3.654478
93,1,73,1003,33.485556,-86.915,1989-06-03,0.002,Alabama,Jefferson,"(1, 73, 1003)",0.016875,0.029500,1073,2.0,1989-06-03,294.0,194.0,0.0,33.0,1989,6,NaN,NaN,NaN,NaN,0.0,10.5,0.0,0.0,1.0,0.0,0.0,12.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1989-06-02,311.0,200.0,0.0,79.0,-4.081922,-3.523365
94,1,73,1003,33.485556,-86.915,1989-06-04,0.006,Alabama,Jefferson,"(1, 73, 1003)",0.016250,0.027875,1073,2.0,1989-06-04,272.0,194.0,0.0,292.0,1989,6,NaN,NaN,NaN,NaN,0.0,10.5,0.0,0.0,1.0,0.0,0.0,12.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1989-06-03,294.0,194.0,0.0,33.0,-4.119662,-3.580025
95,1,73,1003,33.485556,-86.915,1989-06-05,0.012,Alabama,Jefferson,"(1, 73, 1003)",0.015333,0.018250,1073,2.0,1989-06-05,272.0,194.0,0.0,221.0,1989,6,NaN,NaN,NaN,NaN,0.0,10.5,0.0,0.0,1.0,0.0,0.0,12.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,3.0,1989-06-04,272.0,194.0,0.0,292.0,-4.177726,-4.003590


# Full Regression 1

# Simplifying Assumption : Only use states as fixed effects, not individual 

In [80]:
# DD regression 1
dependent_vars = ["ln_ozone", "ln_8hr_ozone"]
treatment_vars = ["treat_rvpI", "treat_rvpII", "treat_rfg", "treat_CARB"]
independent_vars = ["year", "census_region", "State Code"]
control = summer_only.copy()[independent_vars]

In [81]:
# creating census interacted with year for the first DD regression
control["year"] = control["year"].astype(str) 
control["census_region"] = control["census_region"].astype(int).astype(str) 
control["State Code"] = control["State Code"].astype(str) 
control["census_region_year_interaction"] = control["census_region"].str.cat(control["year"], sep=' + ')

In [82]:
control = pd.get_dummies(control, drop_first = True)

In [122]:
results = {}
for i in range(len(dependent_vars)):
    
    dependent_var = dependent_vars[i]
    print("working on dependent, " + dependent_var)
    results[dependent_var] = {}
    
    for j in range(len(treatment_vars)):
        
        treatment_var = treatment_vars[j]
        print("working on treatment, " + treatment_var)
        
        df = pd.concat([summer_only[dependent_var], summer_only[treatment_var], control], axis = 1)
        df = df.dropna(axis = 0, how = "any")

        y = df[dependent_var]
        X = df.drop(columns = [dependent_var])
        model = sm.OLS(y, X).fit()
        
        values = {"Point Estimate":model.params[treatment_var], 
                  "Standard Error":model.bse[treatment_var],
                  "P-Value":model.pvalues[treatment_var],
                  "R Squared":model.rsquared}
        results[dependent_var][treatment_var] = values


working on dependent, ln_ozone
working on treatment, treat_rvpI
working on treatment, treat_rvpII
working on treatment, treat_rfg
working on treatment, treat_CARB
working on dependent, ln_8hr_ozone
working on treatment, treat_rvpI
working on treatment, treat_rvpII
working on treatment, treat_rfg
working on treatment, treat_CARB


In [127]:
result_df_1 = pd.DataFrame(results["ln_ozone"])
result_df_1 = result_df_1.applymap(lambda x: round(x, 4))
result_df_1 = result_df_1.T
index1 = result_df_1.index
index1.name = "ln_ozone"
result_df_1

,Point Estimate,Standard Error,P-Value,R Squared
ln_ozone,,,,
treat_rvpI,-0.0011,0.0037,0.7654,0.1318
treat_rvpII,-0.0197,0.0014,0.0000,0.1321
treat_rfg,-0.0521,0.0015,0.0000,0.1331
treat_CARB,-0.0778,0.0031,0.0000,0.1325


In [126]:
result_df_2 = pd.DataFrame(results["ln_8hr_ozone"])
result_df_2 = result_df_2.applymap(lambda x: round(x, 4))
result_df_2 = result_df_2.T
index2 = result_df_2.index
index2.name = "ln_8hr_ozone"
result_df_2

,Point Estimate,Standard Error,P-Value,R Squared
ln_8hr_ozone,,,,
treat_rvpI,0.0031,0.0036,0.3938,0.1167
treat_rvpII,-0.0031,0.0013,0.0136,0.1170
treat_rfg,-0.0302,0.0014,0.0000,0.1173
treat_CARB,-0.0868,0.0029,0.0000,0.1175
